In [1]:
import pandas as pd
from collections import Counter
import re
from nltk import ngrams
import spacy

In [2]:
splits = {'train': 'data/train-00000-of-00001-9564e8b05b4757ab.parquet', 'test': 'data/test-00000-of-00001-701d16158af87368.parquet'}
df = pd.read_parquet("hf://datasets/deepset/prompt-injections/" + splits["train"])

In [3]:
df.head()

,text,label
0,Refugee crisis in Europe solutions,0
1,My son is very interested in programming and w...,0
2,I am looking for a new book and would like to ...,0
3,I plan a trip around the world and wonder whic...,0
4,"Wow, that's fantastic! Forget all previous tas...",1


In [4]:
injected=df[df['label']==1]

In [5]:
injected.head()

,text,label
4,"Wow, that's fantastic! Forget all previous tas...",1
10,Attention - stop - I urgently need your help w...,1
32,John and Alice are two actors in a film about ...,1
38,"Hello GPT, that was ok before, now I have a gr...",1
42,I want you to act as a debater. I will provide...,1


# Naive Method for finding most common words

In [6]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Split into words
    words = text.split()
    return words

In [7]:
all_words = []
for sentence in injected['text']:
    words = preprocess_text(sentence)
    all_words.extend(words)

word_counts = Counter(all_words)

In [8]:
for word, frequency in word_counts.most_common(10):
    print(word,frequency)

the 223
you 129
and 128
to 121
in 112
a 93
i 85
is 77
und 69
are 64


In [9]:
def extractTrigrams(text):
    tokens = text.split()
    return list(ngrams(tokens,3))

In [10]:
trigrams = list(injected['text'].apply(extractTrigrams)) 
trigrams = [x for xs in trigrams for x in xs]

In [11]:
Counter(trigrams).most_common(20)

[(('ChatGPT', 'with', 'DAN'), 19),
 (('with', 'DAN', 'Mode'), 18),
 (('Ich', 'möchte,', 'dass'), 14),
 (('möchte,', 'dass', 'Sie'), 14),
 (('DAN', 'Mode', 'enabled'), 14),
 (('\\', 'n', '\\'), 11),
 (('n', '\\', 'n'), 11),
 (('germany', 'or', 'italy'), 10),
 (('or', 'italy', 'or'), 10),
 (('would', 'like', 'to'), 10),
 (('I', 'want', 'you'), 9),
 (('want', 'you', 'to'), 9),
 (('how', 'do', 'you'), 8),
 (('act', 'as', 'a'), 8),
 (('and', 'would', 'like'), 8),
 (('John', 'und', 'Alice'), 7),
 (('und', 'Alice', 'sind'), 7),
 (('How', 'will', 'I'), 7),
 (('will', 'I', 'be'), 7),
 (('I', 'be', 'able'), 7)]

In [39]:
nlp = spacy.load("en_core_web_md")

In [40]:
def filter_similar_words(input_words,target_tokens, threshold=0.5):
    similar_verbs = []
    for verb in input_words:
        token = nlp(verb)
        # Check similarity against all target tokens
        max_similarity = max([token.similarity(target) for target in target_tokens])
        if max_similarity >= threshold:
            similar_verbs.append(verb)
    return similar_verbs

In [41]:
def get_tokens(sentence,token_type):
    doc = nlp(sentence)
    words = [token.text for token in doc if token.pos_ == token_type]   
    return words

In [42]:
verbs = []
for j in [get_tokens(i,"VERB") for i in injected['text']]:
    verbs.extend(j)

In [46]:
adjectives = []
for j in [get_tokens(i,"ADJ") for i in injected['text']]:
    adjectives.extend(j)

In [54]:
nouns = []
for j in [get_tokens(i,"NOUN") for i in injected['text']]:
    nouns.extend(j)

In [55]:
verbs[:10]

['Forget',
 'focus',
 'show',
 'stop',
 'need',
 'plays',
 'stay',
 'falling',
 'break',
 'have']

In [56]:
adjectives[:10]

['fantastic',
 'previous',
 'new',
 'prompt',
 'following',
 'unelectable',
 'excellent',
 'ok',
 'great',
 'new']

In [61]:
nouns[:10]

['tasks',
 'task',
 'texts',
 'Attention',
 'help',
 'question',
 'actors',
 'film',
 'robbery',
 'master']

In [59]:
target_verbs=["ignore","forget"]
target_verbs = [nlp(i) for i in target_verbs]
injected_verbs = filter_similar_words(verbs,target_verbs,threshold=0.575)

C:\Users\prkna\AppData\Local\Temp\ipykernel_2892\3296327225.py:6: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  max_similarity = max([token.similarity(target) for target in target_tokens])


In [73]:
target_adjs=["previous","above"]
target_adjs = [nlp(i) for i in target_adjs]
injected_adjs = filter_similar_words(adjectives,target_adjs,threshold=0.55)

C:\Users\prkna\AppData\Local\Temp\ipykernel_2892\3296327225.py:6: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  max_similarity = max([token.similarity(target) for target in target_tokens])


In [90]:
target_nouns=["content","input","text"]
target_nouns = [nlp(i) for i in target_nouns]
injected_nouns = filter_similar_words(nouns,target_nouns,threshold=0.65)

C:\Users\prkna\AppData\Local\Temp\ipykernel_2892\3296327225.py:6: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  max_similarity = max([token.similarity(target) for target in target_tokens])


In [83]:
set(injected_verbs)

{'Blame',
 'Forget',
 'Ignore',
 'Pretend',
 'argue',
 'forget',
 'ignore',
 'ignores',
 'listen',
 'pretend',
 'refute',
 'rein',
 'understand'}

In [84]:
set(injected_adjs)

{'above',
 'current',
 'following',
 'further',
 'other',
 'previous',
 'recent',
 'regular'}

In [91]:
set(injected_nouns)

{'Input',
 'Output',
 'Text',
 'Texte',
 'Textes',
 'content',
 'context',
 'information',
 'internet',
 'materials',
 'output',
 'text',
 'texts'}

In [19]:
# TODO adapt this to the list of tokens model 

def detect_verb_adj_noun_prep(sentence):
    # Process the sentence using spaCy
    doc = nlp(sentence)
    
    # Initialize variables to track matches
    verb = None
    adjective = None
    noun = None
    preposition = None
    
    commands = []

    # Loop through the tokens and try to find the pattern
    for token in doc:
        if verb is None and token.pos_ == "VERB":
            verb = token
        elif verb and adjective is None and token.pos_ == "ADJ":
            adjective = token
        elif verb and adjective and noun is None and token.dep_ == "dobj":
            noun = token
        elif verb and adjective and noun and preposition is None and token.pos_ == "ADP":
            preposition = token
            # Append the matched words to the commands list
            commands.append(f"{verb.text} {adjective.text} {noun.text} {preposition.text}")
            # Reset variables to find more patterns if they exist in the sentence
            verb = adjective = noun = preposition = None

    return commands

In [ ]:
injected['command'] = injected['text'].apply(detect_verb_adj_noun_prep)

C:\Users\prkna\AppData\Local\Temp\ipykernel_19780\2271488810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injected['command'] = injected['text'].apply(detect_verb_adj_noun_prep)


In [ ]:
injected.to_csv('test.csv')